In [47]:
include("src/CombinedSubexpressions.jl")

CombinedSubexpressions

In [48]:
@CombinedSubexpressions.cse inv(H) * G + W * (inv(H) + W)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol inv
            2: Symbol H
          typ: Any
        3: Symbol G
      typ: Any
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Symbol W
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol inv
                2: Symbol H
              typ: Any
            3: Symbol W
          typ: Any
      typ: Any
  typ: Any
nothing
Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Symbol ##invH#320
        3: Symbol G
      typ: Any


LoadError: UndefVarError: H not defined

In [35]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

function h(x, y)
    println("h")
    3
end

x = 1

expr = quote
    function foo(x)
        [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
    end
end

quote  # In[35], line 19:
    function foo(x) # In[35], line 20:
        [h(h(f(g(x)),g(x)),i) == i for i = 1:5]
    end
end

In [36]:
CombinedSubexpressions.cacheify!(expr)

quote 
    begin  # In[35], line 19:
        function foo(x)
            ##gx#297 = g(x)
            ##f##gx#297#298 = f(##gx#297)
            ##h##f##gx#297#298##gx#297#299 = h(##f##gx#297#298,##gx#297)
            begin  # In[35], line 20:
                [h(##h##f##gx#297#298##gx#297#299,i) == i for i = 1:5]
            end
        end
    end
end

In [37]:
@CombinedSubexpressions.cse function foo(x)
    [f(x) == i for i in 1:5]
end

begin 
    function foo(x)
        ##fx#300 = f(x)
        begin  # In[37], line 2:
            [##fx#300 == i for i = 1:5]
        end
    end
end


foo (generic function with 1 method)

In [38]:
foo(1)

f


5-element Array{Bool,1}:
 false
  true
 false
 false
 false

In [39]:
@CombinedSubexpressions.cse begin
    [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
end

begin 
    ##gx#301 = g(x)
    ##f##gx#301#302 = f(##gx#301)
    ##h##f##gx#301#302##gx#301#303 = h(##f##gx#301#302,##gx#301)
    begin  # In[39], line 2:
        [h(##h##f##gx#301#302##gx#301#303,i) == i for i = 1:5]
    end
end
g
f
h
h
h
h
h
h


5-element Array{Bool,1}:
 false
 false
  true
 false
 false

In [41]:
@CombinedSubexpressions.cse begin
    x = zeros(5)
    for i in 1:5
        x[i] = f(x[i]) + g(5)
    end
end

begin 
    ##zeros5#306 = zeros(5)
    ##g5#307 = g(5)
    begin  # In[41], line 2:
        x = ##zeros5#306 # In[41], line 3:
        for i = 1:5 # In[41], line 4:
            x[i] = f(x[i]) + ##g5#307
        end
    end
end
Variables:
  #self#::CombinedSubexpressions.#@cse
  expr::Expr
  result::Any

Body:
  begin 
      result::Any = $(Expr(:invoke, LambdaInfo for cacheify!(::Expr), :(CombinedSubexpressions.cacheify!), :(expr))) # line 75:
      (CombinedSubexpressions.println)(result::Any)::Void # line 76:
      # meta: location essentials.jl esc 134
      SSAValue(1) = result::Any
      # meta: pop location
      return (Core._expr)(:escape,SSAValue(1))::Expr
  end::Expr


In [42]:
@CombinedSubexpressions.cse function bar(y)
    x = zeros(5)
    for i in 1:5
        x[i] = f(x[i]) + g(5)
    end
end

begin 
    function bar(y)
        ##zeros5#308 = zeros(5)
        ##g5#309 = g(5)
        begin  # In[42], line 2:
            x = ##zeros5#308 # In[42], line 3:
            for i = 1:5 # In[42], line 4:
                x[i] = f(x[i]) + ##g5#309
            end
        end
    end
end


bar (generic function with 1 method)

In [43]:
@code_warntype bar(1)

Variables:
  #self#::#bar
  y::Int64
  zeros5#308::Array{Float64,1}
  g5#309::Int64
  x::Array{Float64,1}
  #temp#::Int64
  i::Int64

Body:
  begin 
      zeros5#308::Array{Float64,1} = $(Expr(:invoke, LambdaInfo for fill!(::Array{Float64,1}, ::Float64), :(Base.fill!), :((Core.ccall)(:jl_alloc_array_1d,(Core.apply_type)(Core.Array,Float64,1)::Type{Array{Float64,1}},(Core.svec)(Core.Any,Core.Int)::SimpleVector,Array{Float64,1},0,5,0)::Array{Float64,1}), :((Base.box)(Float64,(Base.sitofp)(Float64,0)))))
      # meta: location In[35] g 2
      # meta: location coreio.jl println 5
      SSAValue(4) = (Core.typeassert)(Base.STDOUT,Base.IO)::IO
      # meta: pop location
      (Base.print)(SSAValue(4),"g",'\n')::Void
      # meta: pop location
      g5#309::Int64 = 1
      # meta: location In[42]
      x::Array{Float64,1} = zeros5#308::Array{Float64,1} # line 3:
      SSAValue(6) = (Base.select_value)((Base.sle_int)(1,5)::Bool,5,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#::Int